In [6]:
%load_ext autoreload
%autoreload 2

In [360]:
from farms import Farms
from user import User
from mocks import Router, Treasury, ERC20

In [363]:
tokens_names = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
tokens = [ERC20(name) for name in tokens_names]

In [364]:
point_list = sorted(list(range(100,1400, 100)) * 2, reverse=True) # [1300, 1300, 1200, 1200, ..., 100, 100]

prices = {tok: price for tok, price in zip(tokens, point_list)}

In [365]:
farms = Farms(1000 * 1e18, 100)

In [367]:
router, treasury, doku = Router(farms, prices), Treasury(), ERC20('doku')

In [368]:
farms.initialize(doku, treasury, router)

In [369]:
users_names= ['alice', 'bob', 'carol', 'david', 
             'eva', 'finn', 'gleb', 'harry', 
             'ivan', 'jack', 'kevin', 'larry', 
             'mary', 'nick', 'oliver', 'peter', 
             'quill', 'rug', 'steve', 'tarry', 
             'ulia', 'vasy', 'will', 'xavier', 
             'yair', 'zoe']

users = [User(name, farms) for name in users_names]

In [370]:
farms.setToken(tokens[0], 100)
farms.setToken(tokens[1], 1000)
farms.setToken(tokens[2], 200)
farms.setToken(tokens[3], 300)
farms.setToken(tokens[4], 100)

In [371]:
sorted(list(range(100,1400, 100)) * 2, reverse=True)

[1300,
 1300,
 1200,
 1200,
 1100,
 1100,
 1000,
 1000,
 900,
 900,
 800,
 800,
 700,
 700,
 600,
 600,
 500,
 500,
 400,
 400,
 300,
 300,
 200,
 200,
 100,
 100]

In [372]:
users[0].mintTokens(tokens[:3], [100, 100, 100])
users[1].mintTokens(tokens[:2], [100, 100])

In [201]:
farms.createPool(users[0], tokens[:3], [33, 33, 33], [100, 100, 100])

In [202]:
farms.deposit(users[1], 0, tokens[:2], [100, 100])

In [204]:
router.getPools()

[{'amounts': {'A': 200, 'B': 200, 'C': 100},
  'weights': {'A': 33, 'B': 33, 'C': 33},
  'users': {'alice': {'amounts': {'A': 100, 'B': 100, 'C': 100}},
   'bob': {'amounts': {'A': 100, 'B': 100}}}}]

In [182]:
farms.block += 100

In [165]:
farms.massUpdateTokens()

In [166]:
farms.harvest(users[0], 0)

In [218]:
0 not in [1,2,3]

True

In [167]:
farms.block += 100

In [185]:
farms.getTokenInfo()

{'A': {'allocPoint': 100,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 200},
 'B': {'allocPoint': 1000,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 200},
 'C': {'allocPoint': 200,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 100},
 'D': {'allocPoint': 300,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 0},
 'E': {'allocPoint': 100,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 0}}

In [186]:
farms.withdrawAndHarvest(users[1], 0, tokens[:2], [100, 100])

In [187]:
farms.getTokenInfo()

{'A': {'allocPoint': 100,
  'lastRewardBlock': 100,
  'accDokuPerShare': 2.941176470588235e+31,
  'amount': 100},
 'B': {'allocPoint': 1000,
  'lastRewardBlock': 100,
  'accDokuPerShare': 2.9411764705882348e+32,
  'amount': 100},
 'C': {'allocPoint': 200,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 100},
 'D': {'allocPoint': 300,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 0},
 'E': {'allocPoint': 100,
  'lastRewardBlock': 0,
  'accDokuPerShare': 0,
  'amount': 0}}

In [92]:
farms.userInfo

{0: {'alice': {'amounts': {'A': 100, 'B': 100, 'C': 100},
   'rewardDebt': 4.411764705882352e+34},
  'bob': {'amounts': {'A': 0, 'B': 0}, 'rewardDebt': 6.470588235287647e+34}}}

In [337]:
n = 2

In [338]:
x, y, z, m = 140, 200, 10, 300
a, b, c, d = 0.2, 0.3, 0.3, 0.2

In [339]:
k = x**a * y**b * z**c * m**d

In [340]:
new_x = x * n**a / n
new_y = y * n**a
new_z = z * n**a
new_m = m * n**a

In [341]:
new_x, new_y, new_z, new_m

(80.40888484979246, 229.73967099940702, 11.48698354997035, 344.6095064991105)

In [342]:
new_x**a * new_y**b * new_z**c * new_m**d

82.21647578478938

In [343]:
k

82.21647578478938

In [356]:
sum_ = new_x * a + new_y * b + new_z * c + new_m * d
sum_

178.4340135427953

In [349]:
n = 2

x, y = 100, 100
a, b = 0.5, 0.5

k = x**a * y**b

new_x = x / (n**(1-a))
new_y = y * n /  (n**(1-a))

In [350]:
new_x**a * new_y**b

99.99999999999999

In [351]:
k

100.0

In [352]:
new_x, new_y

(70.71067811865474, 141.42135623730948)

In [355]:
(new_x * 2 + new_y * 1) * a

141.42135623730948

In [330]:
n = 2

x, y = 100, 100
a, b = 0.5, 0.5

k = x**a * y**b

sum_ = 100 * 2 + 100 * 1

new_x, new_y = sum_ * a, sum_ * b
new_x, new_y

(150.0, 150.0)